## Tg comedy dataset

In [9]:
import os
from glob import glob
import re
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

### Create .txt files

In [31]:
from pathlib import Path
for i in range(80, 90):
    Path(f'comedy-news-tg-dataset/jokes{i}.txt').touch()
    print(f'jokes{i}.txt')

jokes80.txt
jokes81.txt
jokes82.txt
jokes83.txt
jokes84.txt
jokes85.txt
jokes86.txt
jokes87.txt
jokes88.txt
jokes89.txt


### Convert .txt files to .tsv

In [32]:
df = pd.DataFrame()
pattern = r"(.+)\,\s\[.*\]\:\n(.+)"

for filepath in glob('comedy-news-tg-dataset/jokes*.txt'):
    
    if os.path.getsize(filepath) == 0:
        continue
        
    filenum = re.search('\d+', filepath)[0]
    print(filepath, end=', size: ')
    
    with open(filepath) as f:
        setup = f.readline().replace('\xa0', ' ').strip()[:-1]
        
        raw_jokes = f.read().replace('\xa0', ' ').strip()
        raw_jokes = re.sub(r'(Н|н)овость(\W|\n)*', '', raw_jokes)
        raw_jokes = re.sub(f'{setup}(\W|\n)*', '', raw_jokes)
        
    parsed = re.findall(pattern, raw_jokes)
    temp_df = pd.DataFrame(parsed, columns=['author', 'punch'])
    temp_df['setup'] = setup
    temp_df['mark'] = None
    
    temp_df = temp_df[['setup', 'author', 'punch', 'mark']]
    
    temp_df.to_csv(f'comedy-news-tg-dataset/tg_{filenum}.tsv', sep='\t', index=False)
    
    print(temp_df.shape[0])
    df = df.append(temp_df)

comedy-news-tg-dataset/jokes79.txt, size: 128
comedy-news-tg-dataset/jokes78.txt, size: 123
comedy-news-tg-dataset/jokes57.txt, size: 165
comedy-news-tg-dataset/jokes80.txt, size: 134
comedy-news-tg-dataset/jokes81.txt, size: 134
comedy-news-tg-dataset/jokes56.txt, size: 141
comedy-news-tg-dataset/jokes68.txt, size: 141
comedy-news-tg-dataset/jokes83.txt, size: 133
comedy-news-tg-dataset/jokes82.txt, size: 102
comedy-news-tg-dataset/jokes69.txt, size: 126
comedy-news-tg-dataset/jokes64.txt, size: 198
comedy-news-tg-dataset/jokes70.txt, size: 111
comedy-news-tg-dataset/jokes58.txt, size: 135
comedy-news-tg-dataset/jokes59.txt, size: 178
comedy-news-tg-dataset/jokes71.txt, size: 127
comedy-news-tg-dataset/jokes65.txt, size: 137
comedy-news-tg-dataset/jokes73.txt, size: 137
comedy-news-tg-dataset/jokes67.txt, size: 130
comedy-news-tg-dataset/jokes66.txt, size: 132
comedy-news-tg-dataset/jokes72.txt, size: 131
comedy-news-tg-dataset/jokes76.txt, size: 102
comedy-news-tg-dataset/jokes62.txt

### Full dataset creation

In [33]:
# glob.glob('data/comedy-news-tg-dataset/*.tsv')
get_filenum = lambda t: int(re.search('\d+', t)[0])

In [34]:
df = pd.DataFrame()
for filepath in glob('comedy-news-tg-dataset/marked/tg_*.tsv'):
    temp_df = pd.read_csv(filepath, sep='\t')
    temp_df['setup_num'] = get_filenum(filepath)
    temp_df['punch_num'] = temp_df.index + 1
    
    df = df.append(temp_df)
    
df['mark'] = df['mark'].fillna(0).astype(int)
print(df.shape)

(8547, 6)


In [35]:
df.to_csv('comedy-news-tg-dataset/marked/full_dataset.tsv', sep='\t', index=False)

In [36]:
df

,setup,author,punch,mark,setup_num,punch_num
0,Семилетняя россиянка Анастасия Радзинская вошл...,Александр Головко,Ее полное имя Анастасия 1Хbet Радзинская,13,22,1
1,Семилетняя россиянка Анастасия Радзинская вошл...,Хабиб,Лучший родительский стартап,1,22,2
2,Семилетняя россиянка Анастасия Радзинская вошл...,Ксения Журавлева,И получила по жопе,2,22,3
3,Семилетняя россиянка Анастасия Радзинская вошл...,Albert,В десятку за семилетку - такой программе даже ...,5,22,4
4,Семилетняя россиянка Анастасия Радзинская вошл...,Комар,Ее продюсеру за такое продвижение тоже светит ...,2,22,5
...,...,...,...,...,...,...
171,Balenciaga выпустил кроссовки на каблуках почт...,Никита Ватолин,"Фабрика ""Сударь"" в ответ традиционно ничего не...",1,13,172
172,Balenciaga выпустил кроссовки на каблуках почт...,Elizabeth 🦦,Миллениалы изобрели сапоги,0,13,173
173,Balenciaga выпустил кроссовки на каблуках почт...,Комар,За эти шпили у Лили будут шпилли-вилли,1,13,174
174,Balenciaga выпустил кроссовки на каблуках почт...,Aleksandr Scheglov,"Сразу видно, создатели Balenciaga досмотрели м...",0,13,175


## English datasets

In [123]:
import json
import pandas as pd
import re
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
tqdm.pandas()

from deep_translator import GoogleTranslator
from deep_translator.exceptions import NotValidPayload

In [114]:
translator = GoogleTranslator(source='en', target='ru')
def translate_eng2rus(text):
    translated = ''
    try:
        translated =  translator.translate(text)
    except NotValidPayload:
        print(f'Broken text: {text}')
    
    return translated

In [132]:
def remove_special_tokens(text):
    result = ''
    try:
        result = re.sub(r'\r|\n|\t', '', text)
    except:
        pass
    return result

### Reddit

In [105]:
with open('reddit_jokes.json') as f:
    reddit_dict = json.load(f)

In [111]:
reddit_df = pd.read_csv('/Users/naltukhov/Documents/programs/jokes-generator/eng-joke-dataset/reddit-dataset.tsv',
                        sep='\t')

/Users/naltukhov/Documents/programs/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [117]:
while True:
    need_translate_df = reddit_df.loc[reddit_df['rus_punch'].isna()]
    
    if len(need_translate_df) == 0:
        break
        
    print(len(need_translate_df))
    
    need_translate_index_sample = need_translate_df.iloc[0: min(1000, len(need_translate_df))].index
    
    reddit_df.loc[need_translate_index_sample, 'rus_punch'] = \
        reddit_df.loc[need_translate_index_sample, 'punch'].progress_apply(translate_eng2rus)
    
    reddit_df.loc[need_translate_index_sample, 'rus_setup'] = \
        reddit_df.loc[need_translate_index_sample, 'setup'].progress_apply(translate_eng2rus)

  0%|          | 0/1000 [00:00<?, ?it/s]

1185056


  0%|          | 0/1000 [00:00<?, ?it/s]

Broken text: nan
Broken text: nan
Broken text: nan
Broken text: nan


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Broken text: nan


  0%|          | 2/1000 [00:00<00:53, 18.54it/s]

1142062


 78%|███████▊  | 781/1000 [19:29:25<5:27:55, 89.84s/it]


KeyboardInterrupt: 

In [118]:
reddit_df.head()

,punch,mark,setup,rus_punch,rus_setup
0,On the condition he gets to install windows.,48526,Breaking News: Bill Gates has agreed to pay fo...,"При условии, что он установит окна.",Срочные новости: Билл Гейтс согласился заплати...
1,/r/Jokes,45500,I found a place where the recycling rate is 98%,/ г / Шутки,"Я нашел место, где уровень утилизации составля..."
2,"But its a silly comparison really, its like co...",39570,Steve jobs would have been a better president ...,"Но на самом деле это глупое сравнение, это как...","Стив Джобс был бы лучшим президентом, чем Дона..."
3,We went and had some drinks. Cool guy. Wants t...,36421,My girlfriend told me to take the spider out i...,Мы пошли и выпили. Клевый парень. Хочет быть в...,"Моя девушка посоветовала мне вытащить паука, а..."
4,Please don't upvote. Her strap-on is huge.,35772,"For every upvote this gets, my girlfriend and ...","Пожалуйста, не голосуйте. Ее страпон огромен.",На каждый положительный голос мы с моей девушк...


In [119]:
reddit_df.to_csv('/Users/naltukhov/Documents/programs/jokes-generator/eng-joke-dataset/reddit-dataset.tsv',
                 sep='\t', index=False)

In [135]:
rus_reddit_df = reddit_df.loc[~reddit_df.rus_punch.isna(), 
                              ['rus_punch', 'rus_setup', 'mark']]
rus_reddit_df.columns = ['punch', 'setup', 'mark']

rus_reddit_df.punch = rus_reddit_df.punch.apply(remove_special_tokens)
rus_reddit_df.setup = rus_reddit_df.setup.apply(remove_special_tokens)

In [137]:
rus_reddit_df

,punch,setup,mark
0,"При условии, что он установит окна.",Срочные новости: Билл Гейтс согласился заплати...,48526
1,/ г / Шутки,"Я нашел место, где уровень утилизации составля...",45500
2,"Но на самом деле это глупое сравнение, это как...","Стив Джобс был бы лучшим президентом, чем Дона...",39570
3,Мы пошли и выпили. Клевый парень. Хочет быть в...,"Моя девушка посоветовала мне вытащить паука, а...",36421
4,"Пожалуйста, не голосуйте. Ее страпон огромен.",На каждый положительный голос мы с моей девушк...,35772
...,...,...,...
44990,"Те, что писают в душе, и лжецы!",В жизни есть два типа людей,11
44991,"Это объясняет, что случилось с теми килограмма...","[Проблема решена] Говорят, когда вы бреете вол...",11
44992,Мокрый нос.,В чем разница между близоруким и дальнозорким ...,11
44993,Вытащил мидию!,Только что вернулся с дискотеки с морепродуктами.,11


In [138]:
rus_reddit_df.to_csv('/Users/naltukhov/Documents/programs/jokes-generator/eng-joke-dataset/reddit-rus-dataset.tsv',
                 sep='\t', index=False)

### Wocka

In [19]:
with open('wocka.json') as f:
    wocka_dict = json.load(f)

In [20]:
wocka_dict[2]

{'body': "What's black and white and red all over?\r\n\r\nA newspaper.",
 'category': 'Other / Misc',
 'id': 4,
 'title': 'Black, White and Red'}

In [25]:
wocka_df = pd.DataFrame(wocka_dict) \
    .drop(['id', 'title'], axis=1) \
    .rename(columns={'body': 'punch'}) 

wocka_df.punch = wocka_df.punch.str.replace('\n', '').replace('\t', '')

# Specify len < 500
wocka_df = wocka_df.loc[wocka_df.punch.apply(len) <= 500]
wocka_df = wocka_df.loc[wocka_df.punch.apply(len) > 5]

wocka_df['rus_punch'] = None

In [ ]:
wocka_df = pd.read_csv('/Users/naltukhov/Documents/programs/jokes-generator/eng-joke-dataset/wocka-dataset.tsv',
                        sep='\t')

In [ ]:
while True:
    need_translate_df = wocka_df.loc[wocka_df['rus_punch'].isna()]
    
    if len(need_translate_df) == 0:
        break
        
    print(len(need_translate_df))
    
    need_translate_index_sample = need_translate_df.iloc[0: min(1000, len(need_translate_df))].index
    
    wocka_df.loc[need_translate_index_sample, 'rus_punch'] = \
        wocka_df.loc[need_translate_index_sample, 'punch'].progress_apply(translate_eng2rus)

In [ ]:
wocka_df = wocka_df[['rus_punch', 'category']] \
    .rename(columns={'rus_punch': 'joke'})

remove_tokens = lambda text: re.sub(r'\r|\n|\t', '', text)

wocka_df.joke = wocka_df.joke.apply(remove_tokens)

In [ ]:
wocka_df.to_csv('/Users/naltukhov/Documents/programs/jokes-generator/eng-joke-dataset/wocka-dataset.tsv',
                 sep='\t', index=False)

In [28]:
wocka_df

,punch,category
0,What do you call a cow with no legs?\r\n\r\nGr...,Animal
1,What do you call a cow jumping over a barbed w...,Animal
2,What's black and white and red all over?\r\n\r...,Other / Misc
3,"So, this guy walks into a bar.\r\n\r\nAnd says...",Bar
4,"If the opposite of pro is con, isn't the oppos...",One Liners
...,...,...
10014,(A man comes to my register with a mint chocol...,Men / Women
10015,(I am shelving DVDs in a library when a man co...,Children
10016,"Me: ""That will be 17.50, please.""\r\n\r\nCusto...",Religious
10017,"Me: ""Sir, would you like to use any coupons to...",At Work


## Twitter dataset

In [2]:
import pandas as pd
import json

In [3]:
with open('tweets-dataset/train.json') as f:
    train_json = json.load(f)

In [22]:
with open('tweets-dataset/test.json') as f:
    test_json = json.load(f)

In [25]:
df = pd.concat([
    pd.DataFrame(train_json.items(), columns=['joke', 'is_joke']),
    pd.DataFrame(test_json.items(), columns=['joke', 'is_joke']),
], axis=0)
df = df.loc[df.is_joke == 1][['joke']]

In [26]:
df.to_csv('tweets-dataset/tweeter_jokes.tsv', sep='\t', index=False)

In [27]:
df.shape

(156605, 1)

In [28]:
df.sample()

,joke
68637,"- Скажите, Холмс, а почему в Сочи не приедет О..."


## Anecdote dataset

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('anecdote-dataset/anecdote_dataset.tsv', sep='\t')

In [31]:
df

,joke_text,likes
0,- Все проблемы в Думе решит Максим.\n- Это кто...,24
1,"Муж с женой в постели. Жена спит, мужу не спит...",17
2,Никогда не забывай свои корни! Постоянно закра...,18
3,"- Скажите, вы - натуральная блондинка?\n- Ммм....",12
4,Ходил с друзьями на курсы по избавлению от мат...,11
...,...,...
82895,Купи пять беляшей и собери кошку!,5
82896,"Терапевт с неразборчивым почерком, выписывая р...",3
82897,"Друг не тот, кто помог подняться, а тот, кто н...",12
82898,Сегодня я расскажу вам о высшем образовании.\n...,4


## Full jokes dataset

In [172]:
import pandas as pd
import re

In [186]:
anecdote_df = pd.read_csv('anecdote-dataset/anecdote_dataset.tsv', sep='\t') \
    .rename(columns={'joke_text': 'joke'})[['joke']]
kvn_df = pd.read_csv('kvn-dataset/kvn_dataset.tsv', sep='\t') \
    .rename(columns={'joke_text': 'joke'})[['joke']]
wocka_df = pd.read_csv('eng-joke-dataset/wocka-dataset.tsv', sep='\t') \
    [['joke']]
tweets_df = pd.read_csv('tweets-dataset/tweeter_jokes.tsv', sep='\t')

In [188]:
full_joke_df = pd.concat([anecdote_df, kvn_df, wocka_df, tweets_df], axis=0) \
    .reset_index(drop=True)
full_joke_df.shape

(246771, 1)

### Clean

In [189]:
def remove_special_tokens(text):
    result = ''
    try:
        result = re.sub(r'\r|\n|\t', '', text)
    except:
        pass
    return result

full_joke_df.joke = full_joke_df.joke.apply(remove_special_tokens)
full_joke_df = full_joke_df.drop_duplicates(subset=['joke'])
full_joke_df.shape

(241553, 1)

In [190]:
full_joke_df.sample(5)

,joke
181853,"Артисты - это профессиональные обманщики, но к..."
148164,"Для тех, кому ещё не поздно одуматься: - Один ..."
56100,А я не буду на Новый год смотреть Голубой огон...
161596,"Кого собираются засудить родители детей, расст..."
13224,- Как вас подстричь?- Сначала выравниваете вер...


### Try to find setup and punch in full joke

In [191]:
def setup_punch_split(text):
    parts = re.split(r'\.{3}|\.|\?|\!|:', text)
    
    if (len(parts) > 3) or (len(parts) < 2):
        return None
    
    if (len(parts[-1]) == 0):
        parts = parts[:-1]
        
    if (len(parts) == 2) and (len(parts[0]) > 10) and (len(parts[1]) > 10):
        return (parts[0].strip(), parts[1].strip())

In [192]:
punch_setup_df = full_joke_df.copy()
punch_setup_df['setup_punch'] = punch_setup_df.joke.apply(setup_punch_split)
punch_setup_df = punch_setup_df.loc[~punch_setup_df.setup_punch.isna()]
punch_setup_df.shape

(62684, 2)

In [193]:
punch_setup_df = pd.DataFrame(punch_setup_df.setup_punch.tolist(), columns=['setup', 'punch'])

In [194]:
# Remove setup-punch from full
full_joke_df = full_joke_df.loc[~full_joke_df.index.isin(punch_setup_df.index)]
full_joke_df.shape

(178869, 1)

In [195]:
# Save
punch_setup_df.to_csv('setup-punch-agg-dataset/setup-punch-from-full-jokes.tsv', sep='\t', index=False)
full_joke_df.to_csv('full-text-agg-dataset/full-text-agg-dataset.tsv', sep='\t', index=False)

## Setup-punch datasets

In [198]:
reddit_df = pd.read_csv('eng-joke-dataset/reddit-rus-dataset.tsv', sep='\t') \
    [['setup', 'punch']]
from_full_df = pd.read_csv('setup-punch-agg-dataset/setup-punch-from-full-jokes.tsv', sep='\t')

In [200]:
setup_punch_df = pd.concat([from_full_df, reddit_df], axis=0)

### Clean

In [201]:
def remove_special_tokens(text):
    result = ''
    try:
        result = re.sub(r'\r|\n|\t', '', text)
    except:
        pass
    return result

setup_punch_df.setup = setup_punch_df.setup.apply(remove_special_tokens)
setup_punch_df.punch = setup_punch_df.punch.apply(remove_special_tokens)

setup_punch_df = setup_punch_df.drop_duplicates(subset=['setup'])
setup_punch_df.shape

(98718, 2)

In [202]:
setup_punch_df.sample(5)

,setup,punch
34719,"Эй, Великобритания! Ты отлично выглядишь!",Вы потеряли несколько фунтов?
17769,Что общего между банджи-джампингом и проститут...,"Они оба дешевы, быстры, и если резина порвется..."
7048,какая разница между конгрессом и презервативом?,В презерватив можно поместить только один член.
41711,"Я стал называть свой пенис ""Локоть""","Потому что это сложно, гибко и неприлично выст..."
16741,Проктолог,Парень идет к своему проктологу на колоноскопи...


In [203]:
setup_punch_df.to_csv('setup-punch-agg-dataset/setup-punch-agg-dataset.tsv', sep='\t', index=False)